<a href="https://colab.research.google.com/github/caiofoti/Hepatitis-RS/blob/main/ScripPainelHepatitesVirais_Atual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Importando bibliotecas: { display-mode: "form" }
!pip install pandas dbfread
!pip install pyreadstat

# Importando bibliotecas necessárias para manipulação e análise de dados
import warnings
import numpy as np
import pandas as pd
from dbfread import DBF
from functools import partial, reduce
from datetime import datetime
from pandas.api.types import CategoricalDtype

# Importando bibliotecas necessárias para visualização de dados
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact

In [ ]:
# @title Selecione os Arquivos: { vertical-output: true, form-width: "10px", display-mode: "form" }
import os
import pandas as pd
from dbfread import DBF
from google.colab import drive
from IPython.display import clear_output

# Função para montar o Google Drive
def mount_drive():
    drive.mount('/content/gdrive/', force_remount=True)

# Função para listar todos os arquivos na pasta
def list_files_in_folder(folder_path):
    file_list = os.listdir(folder_path)
    file_list.sort()
    return file_list

# Função para importar um arquivo específico a partir do nome e atribuir um nome personalizado ao dataframe global
def import_and_assign_dataframe(folder_path, file_name, dataframe_name, sheet_name=None):
    file_path = os.path.join(folder_path, file_name)

    try:
        if file_name.lower().endswith('.dbf'):
            table = DBF(file_path)
            df = pd.DataFrame(iter(table))
        elif file_name.lower().endswith(('.xlsx', '.xls', '.ods', '.csv')):
            if sheet_name is None:
                df = pd.read_excel(file_path)
            else:
                df = pd.read_excel(file_path, sheet_name=sheet_name)
        else:
            raise ValueError(f'Formato de arquivo não suportado: {file_name}')

        globals()[dataframe_name] = df  # Atribuir o dataframe global com o nome personalizado
        print(f'Dataframe "{file_name}" importado como "{dataframe_name}" com sucesso.')
    except FileNotFoundError:
        print(f'Arquivo "{file_name}" não encontrado.')
    except Exception as e:
        print(f'Erro ao importar o arquivo "{file_name}": {str(e)}')

# Função principal
def main():
    # Montar o Google Drive
    mount_drive()

    # Defina o caminho da pasta no seu Google Drive
    folder_path = '/content/gdrive/MyDrive/Painel Hepatites Viras - Google Data Studio/Banco HEPANET'
    #folder_path = '/content/gdrive/MyDrive/Colab Data Management'
    # Listar todos os arquivos na pasta
    file_list = list_files_in_folder(folder_path)

    print('Lista de arquivos na pasta:')
    for i, file in enumerate(file_list):
        print(f'{i + 1}: {file}')

    while True:
        selection = input('Selecione o número do arquivo que deseja importar (ou "Q" para sair): ').strip().lower()

        if selection == 'q':
            break

        try:
            selection = int(selection)
            if 1 <= selection <= len(file_list):
                selected_file_name = file_list[selection - 1]
                dataframe_name = input('Digite o nome para o dataframe global: ')
                sheet_name = None

                if selected_file_name.lower().endswith(('.xlsx', '.xls', '.ods', '.csv')):
                    sheet_name = input('Digite o nome da planilha (deixe em branco para usar a padrão): ').strip()

                import_and_assign_dataframe(folder_path, selected_file_name, dataframe_name, sheet_name)
            else:
                print('Seleção inválida. Tente novamente.')

        except ValueError:
            print('Seleção inválida. Tente novamente.')

# Chamar a função principal
main()


Mounted at /content/gdrive/
Lista de arquivos na pasta:
1: BancoHep_1311.DBF
2: BancoHep_1710.DBF
3: BancoHep_2710.DBF
4: base_procv.xlsx
Selecione o número do arquivo que deseja importar (ou "Q" para sair): 1
Digite o nome para o dataframe global: dbf
Dataframe "BancoHep_1311.DBF" importado como "dbf" com sucesso.
Selecione o número do arquivo que deseja importar (ou "Q" para sair): 4
Digite o nome para o dataframe global: base_procv
Digite o nome da planilha (deixe em branco para usar a padrão): reduzido
Dataframe "base_procv.xlsx" importado como "base_procv" com sucesso.
Selecione o número do arquivo que deseja importar (ou "Q" para sair): base_pop
Seleção inválida. Tente novamente.
Selecione o número do arquivo que deseja importar (ou "Q" para sair): completo
Seleção inválida. Tente novamente.
Selecione o número do arquivo que deseja importar (ou "Q" para sair): 4
Digite o nome para o dataframe global: base_pop
Digite o nome da planilha (deixe em branco para usar a padrão): complet

In [ ]:
#Copiando o DBF para separar o arquivo fonte do que iremos manipular.
hepatite = dbf.copy()

---

In [ ]:
# @title Filtro inicial para definição dos casos de hepatite: { vertical-output: true, form-width: "50px" }

# Definir start_date como a data inicial
start_date = pd.to_datetime('2015-01-01')

# Encontrar a data máxima no DataFrame para end_date
end_date = hepatite['DT_SIN_PRI'].max()

# Filtrar o DataFrame usando start_date e end_date
hepatite = hepatite.loc[(hepatite['SG_UF'] == '43') & (hepatite['DT_SIN_PRI'] >= start_date) & (hepatite['DT_SIN_PRI'] <= end_date)]

# Verificando se há dados no DataFrame após o filtro
if hepatite.empty:
    print("Nenhum dado após o filtro. Encerrando o processo.")
else:
    # Convertendo a coluna 'DT_SIN_PRI' para o formato datetime
    hepatite['anodiag'] = pd.to_datetime(hepatite['DT_SIN_PRI'])

    # Convertendo a coluna 'anodiag' para o formato ano
    hepatite['anodiag'] = hepatite['anodiag'].dt.year

    # Verificando a presença de cada tipo de hepatite
    hepatite['Hep_A'] = np.where((hepatite['ANTIHAVIGM'] == '1') | ((hepatite['CLASSI_FIN'] == '2') & (hepatite['CLAS_ETIOL'] == '01')), True, False)
    hepatite['Hep_B'] = np.where((hepatite['AGHBS'] == '1') | (hepatite['ANTIHBCIGM'] == '1'), True, False)
    hepatite['Hep_C'] = np.where((hepatite['ANTIHCV'] == '1') | (hepatite['TP_SOROHCV'] == '1'), True, False)

    # Criando um DataFrame com resumo de casos
    hepatite_summary = hepatite.groupby(['anodiag'])[['Hep_A', 'Hep_B', 'Hep_C']].sum()

    # Transpondo a tabela
    hepatite_summary = hepatite_summary.T

    # Imprimindo o resumo
    print("Filtro inicial bem-sucedido - Nenhum erro encontrado.\n")
    print(hepatite_summary)

# Ignorando os SettingWithCopyWarning e FutureWarning
warnings.simplefilter("ignore", category=pd.errors.SettingWithCopyWarning)
warnings.simplefilter("ignore", category=FutureWarning)

Filtro inicial bem-sucedido - Nenhum erro encontrado.

anodiag  2015  2016  2017  2018  2019  2020  2021  2022  2023
Hep_A      71    77    78   151    59    36    26   133   294
Hep_B    1617  1491  1411  1509  1498   897   954  1070   807
Hep_C    5425  5294  4864  5045  4778  2605  2671  2838  2339




---



In [ ]:
# @title Compatibilizando os ID's dos municípios de residência com a base geográfica de municípios. { vertical-output: true }
try:
    # Tornando as variáveis dos dois datasets compatíveis.

    # Checking the class of ID_MN_RESI column in HC_menor dataframe
    type(hepatite['ID_MN_RESI'])

    # Checking the class of cod_ibge column in base_procv dataframe
    type(base_procv['cod_ibge'])

    # Converting cod_ibge column to integer in base_procv dataframe
    base_procv['cod_ibge'] = base_procv['cod_ibge'].fillna(-1).astype(int)

    # Converting ID_MN_RESI column to integer in HC_menor dataframe
    hepatite['ID_MN_RESI'] = hepatite['ID_MN_RESI'].fillna(-1).astype(int)

    # Unindo as informações da tabela base_procv com a tabela hepatites.
    hepatite = pd.merge(
        # Unindo a tabela 'hepatites'
        left=hepatite,
        # Com a tabela 'base_procv'
        right=base_procv,
        # Selecionando as colunas 'ID_MN_RESI' e 'cod_ibge' para unir os bancos de dados.
        left_on='ID_MN_RESI',
        right_on='cod_ibge',
        # Type of join
        how='left'
    )
    print("Merge bem-sucedido, nenhum erro encontrado.")
except Exception as e:
    print(f"Ocorreu um erro durante o merge: {str(e)}")

# Verificar se o merge foi bem-sucedido
if len(hepatite) > 0:
    print(f"Foram encontradas {len(hepatite)} linhas após o merge.")
else:
    print("O merge não gerou linhas correspondentes.")

Merge bem-sucedido, nenhum erro encontrado.
Foram encontradas 61475 linhas após o merge.




---



In [ ]:
# @title Agregando casos de hepatite por município.
try:
    # Contabilizando e agrupando os casos
    hepatite['tipo_hepatite'] = np.where(hepatite['Hep_A'], 'Hepatite A',
                                         np.where(hepatite['Hep_B'], 'Hepatite B',
                                                  np.where(hepatite['Hep_C'], 'Hepatite C',
                                                           'Outro'))
                                         )

    # Agrupando os dados por município e ano
    hepatite_agrupada = hepatite.groupby([
        'tipo_hepatite', 'anodiag', 'name', 'cod_ibge','cod_crs'
    ]).size().reset_index()

    # Renomeando as colunas
    hepatite_agrupada.columns = [
        'Hepatite', 'Ano', 'Município', 'cod_ibge', 'cod_crs', 'quantidade'
    ]

    # Ordenando os valores
    hepatite_agrupada = hepatite_agrupada.sort_values(
        ['Hepatite', 'Ano', 'Município']
    )

    # Filtra os casos do tipo "Outros"
    hepatite_agrupada = hepatite_agrupada[hepatite_agrupada['Hepatite'] != 'Outro']


    # Verificando se a operação foi bem-sucedida
    if 'quantidade' in hepatite_agrupada.columns:
        print("Operação concluída com sucesso!\n")

        # Exibindo a tabela de contagem e agrupamento
        print(hepatite_agrupada.head())
    else:
        print("A operação falhou. Verifique os dados e a estrutura do DataFrame.")

except Exception as e:
    print(f"Ocorreu um erro durante a contabilização e agrupamento dos casos: {str(e)}")

Operação concluída com sucesso!

     Hepatite   Ano Município  cod_ibge  cod_crs  quantidade
0  Hepatite A  2015  Alvorada    430060        1           1
1  Hepatite A  2015      Bagé    430160        7           1
2  Hepatite A  2015   Camaquã    430350        1           1
3  Hepatite A  2015   Canguçu    430450        3           1
4  Hepatite A  2015    Canoas    430460        1           1




---



In [ ]:
# @title Agregando casos de hepatite por Coordenadorias Regionais de Saúde

# Defina a variável `hepatite_crs` fora da função
hepatite_crs = None

# Defina a função que agregará os casos de hepatite por Coordenadorias Regionais de Saúde
def aggregate_hepatitis_cases(hepatite_agrupada, base_pop):
    try:
        # Filtra os casos do tipo "Outros"
        hepatite_agrupada = hepatite_agrupada[hepatite_agrupada['Hepatite'] != 'Outro']

        # Etapa 1: Groupby do hepatite_agrupada por cod_crs
        hepatite_agrupada_crs = hepatite_agrupada.groupby(['cod_crs', 'Ano', 'Hepatite']).sum().reset_index()

        # Etapa 2: Groupby da base_pop por cod_crs
        base_pop_crs = base_pop.groupby(['cod_crs','crs']).sum().reset_index()

        # Mesclando os DataFrames de casos de hepatite com os quantitativos populacionais por cod_crs
        hepatite_crs = pd.merge(hepatite_agrupada_crs, base_pop_crs, on='cod_crs')

        # Etapa 3: Reorganizando as populações por ano em uma única coluna "pop"
        anos = [f'pop_{ano}' for ano in range(15, 24)]  # Colunas de pop para os anos 2015 a 2023

        # Reorganizando as colunas de pop por ano em uma única coluna "pop"
        hepatite_crs['pop'] = hepatite_crs.apply(
            lambda row: row[f'pop_{str(row["Ano"])[2:]}'], axis=1
        )

        # Removendo as colunas pop_{ano} que não serão mais utilizadas
        hepatite_crs.drop(columns=anos, inplace=True)

        # Calcular a taxa por 100 mil habitantes e adicionar uma coluna
        hepatite_crs['Taxa'] = ((hepatite_crs['quantidade'] / hepatite_crs['pop']) * 100000).round(1)

        # Reordenando as colunas conforme o desejado
        hepatite_crs = hepatite_crs[['Hepatite', 'Ano', 'cod_crs','crs', 'pop', 'quantidade', 'Taxa']]

        # Exibindo a tabela final
        print("Operação concluída com sucesso!\n")
        return hepatite_crs
    except Exception as e:
        print(f"Ocorreu um erro durante a agregação dos casos: {str(e)}")
        return None

# Chamando a função `aggregate_hepatitis_cases()`
hepatite_crs = aggregate_hepatitis_cases(hepatite_agrupada, base_pop)


# Exibindo a tabela final
if hepatite_crs is not None:
    print(hepatite_crs.head())
else:
    print("A função `aggregate_hepatitis_cases()` retornou `None`.")

Operação concluída com sucesso!

     Hepatite   Ano  cod_crs       crs      pop  quantidade  Taxa
0  Hepatite A  2015        1  01Âª CRS  4555610          34   0.7
1  Hepatite B  2015        1  01Âª CRS  4555610         622  13.7
2  Hepatite C  2015        1  01Âª CRS  4555610        3115  68.4
3  Hepatite A  2016        1  01Âª CRS  4566076          39   0.9
4  Hepatite B  2016        1  01Âª CRS  4566076         583  12.8




---



In [ ]:
# @title Casos, Taxa, Mun e RS

# Função para calcular a taxa de incidência por 100 mil habitantes
def calcular_taxa(row):
    return round(row['quantidade'] / row['pop'] * 100000, 1)

# Cria o dataframe "casos_completos" para preencher casos vazios
casos_completos = pd.DataFrame(columns=['Hepatite', 'Ano', 'Município', 'cod_ibge', 'quantidade'])

# Encontra a lista de hepatites e anos únicos
hepatites_unicas = hepatite_agrupada['Hepatite'].unique()
anos_unicos = hepatite_agrupada['Ano'].unique()

for hepatite in hepatites_unicas:
    for ano in anos_unicos:
        for i, municipio in enumerate(base_procv['name']):
            filtro = (hepatite_agrupada['Hepatite'] == hepatite) & (hepatite_agrupada['Ano'] == ano) & (hepatite_agrupada['Município'] == municipio)
            quantidade = hepatite_agrupada.loc[filtro, 'quantidade']

            if len(quantidade) == 1:
                casos_completos = casos_completos.append({
                    'Hepatite': hepatite,
                    'Ano': ano,
                    'Município': municipio,
                    'cod_ibge': base_procv.loc[i, 'cod_ibge'],
                    'quantidade': quantidade.iloc[0]  # Use iloc to acessar o valor
                }, ignore_index=True)
            else:
                casos_completos = casos_completos.append({
                    'Hepatite': hepatite,
                    'Ano': ano,
                    'Município': municipio,
                    'cod_ibge': base_procv.loc[i, 'cod_ibge'],
                    'quantidade': 0  # Preencha com zero quando não houver casos
                }, ignore_index=True)


# Filtra os casos do tipo "Outros"
casos_completos = casos_completos[casos_completos['Hepatite'] != 'Outro']

# Mescla "casos_completos" com "base_pop" para adicionar população
casos_completos = pd.merge(casos_completos, base_pop, on='cod_ibge')

# Etapa 3: Reorganizando as populações por ano em uma única coluna "pop"
anos = [f'pop_{ano}' for ano in range(15, 24)]  # Colunas de pop para os anos 2015 a 2023

# Reorganizando as colunas de pop por ano em uma única coluna "pop"
casos_completos['pop'] = casos_completos.apply(
    lambda row: row[f'pop_{str(row["Ano"])[2:]}'], axis=1
    )

# Removendo as colunas pop_{ano} que não serão mais utilizadas
casos_completos.drop(columns=anos, inplace=True)

# Calcula a taxa por 100 mil habitantes e adiciona uma coluna
casos_completos['Taxa'] = casos_completos.apply(calcular_taxa, axis=1)

# Etapa 1: Groupby do hepatite_agrupada por cod_crs
hepatite_agrupada_RS = hepatite_agrupada.groupby(['Ano', 'Hepatite'])['quantidade'].sum().reset_index()

# Adiciona coluna "cod_ibge" com o valor de "RS" para representar o Rio Grande do Sul
hepatite_agrupada_RS['cod_ibge'] = 'RS'

hepatite_agrupada_RS['Município'] = 'RS'

hepatite_agrupada_RS['abbrev_state'] = 'RS'

hepatite_agrupada_RS['code_muni'] = '43'

# Etapa 2: Groupby da base_pop por cod_crs
base_pop_RS = base_pop.groupby(['abbrev_state']).sum().reset_index()

# Mesclando os DataFrames de casos de hepatite com os quantitativos populacionais por cod_crs
hepatite_RS = pd.merge(hepatite_agrupada_RS, base_pop_RS, on='abbrev_state')

# Etapa 3: Reorganizando as populações por ano em uma única coluna "pop"
anos = [f'pop_{ano}' for ano in range(15, 24)]  # Colunas de pop para os anos 2015 a 2023

# Reorganizando as colunas de pop por ano em uma única coluna "pop"
hepatite_RS['pop'] = hepatite_RS.apply(
    lambda row: row[f'pop_{str(row["Ano"])[2:]}'], axis=1
    )

# Removendo as colunas pop_{ano} que não serão mais utilizadas
hepatite_RS.drop(columns=anos, inplace=True)

# Calcular a taxa por 100 mil habitantes e adicionar uma coluna
hepatite_RS['Taxa'] = hepatite_RS.apply(calcular_taxa, axis=1)

# Reordenando as colunas conforme o desejado
hepatite_RS = hepatite_RS[['Hepatite', 'Ano', 'Município','pop', 'quantidade', 'Taxa']]

casos_completos = pd.concat([casos_completos, hepatite_RS], ignore_index=True)

# Convert the 'Ano' column to a datetime column
casos_completos['Ano'] = pd.to_datetime(casos_completos['Ano'], format='%Y')

# Cria uma nova coluna com o formato "01/01/%Y"
casos_completos['Data_Formatada'] = casos_completos['Ano'].dt.strftime('01/01/%Y')

# Solicita ao usuário que escolha as colunas a serem mantidas
print("Colunas disponíveis:")
for i, col in enumerate(casos_completos.columns):
    print(f"{i}: {col}")

colunas_escolhidas = input("Digite os números das colunas que deseja manter, separadas por vírgula: ").split(',')
colunas_escolhidas = [casos_completos.columns[int(col)] for col in colunas_escolhidas]
casos_completos = casos_completos[colunas_escolhidas]

# Verifica se a operação foi bem-sucedida
if 'Data_Formatada' in casos_completos.columns:
    print("\nOperação concluída com sucesso!")

Colunas disponíveis:
0: Hepatite
1: Ano
2: Município
3: cod_ibge
4: quantidade
5: code_muni
6: name
7: crs
8: crs2
9: cod_crs
10: macro
11: code_state
12: abbrev_state
13: name_state
14: pop
15: Taxa
16: Data_Formatada
Digite os números das colunas que deseja manter, separadas por vírgula: 0,1,2,3,4,5,14,15,16

Operação concluída com sucesso!


In [ ]:
casos_completos.head()

,Hepatite,Ano,Município,cod_ibge,quantidade,code_muni,pop,Taxa,Data_Formatada
0,Hepatite A,2015-01-01,Aceguá,430003,0,4300034.0,4611,0.0,01/01/2015
1,Hepatite A,2016-01-01,Aceguá,430003,0,4300034.0,4595,0.0,01/01/2016
2,Hepatite A,2017-01-01,Aceguá,430003,0,4300034.0,4412,0.0,01/01/2017
3,Hepatite A,2018-01-01,Aceguá,430003,0,4300034.0,4487,0.0,01/01/2018
4,Hepatite A,2019-01-01,Aceguá,430003,0,4300034.0,4516,0.0,01/01/2019


In [ ]:
# @title Salvando no Drive...

# Caminho para salvar os arquivos localmente
local_save_path = "/content/gdrive/MyDrive/Painel Hepatites Viras - Google Data Studio/Alimentação"

# Verificando se o diretório existe
if not os.path.exists(local_save_path):
    # Criando o diretório
    os.makedirs(local_save_path)

# Salvando os arquivos
file_name1 = f"HistogramaPainelHepatitesVirais_Mun_RS.xlsx"
file_path1 = os.path.join(local_save_path, file_name1)
casos_completos.to_excel(file_path1, index=False)

file_name2 = f"HistogramaPainelHepatitesVirais_CRS.xlsx"
file_path2 = os.path.join(local_save_path, file_name2)
hepatite_crs.to_excel(file_path2, index=False)



▶ ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



In [ ]:
# @title Faixa Etária
try:
    # Selecionando variáveis
    HPET_menor = dbf.loc[:, ['DT_NOTIFIC', 'ID_MUNICIP', 'DT_SIN_PRI', 'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA',
                            'CS_ESCOL_N', 'ID_MN_RESI', 'INSTITUCIO', 'HIV', 'COLETAMARC', 'ANTIHAVIGM', 'AGHBS',
                            'ANTIHBCIGM', 'ANTIHCV', 'TP_SOROHCV', 'GEN_VHC', 'CLASSI_FIN', 'FORMA', 'CLAS_ETIOL',
                            'FONTE', 'DT_ENCERRA', 'SG_UF']]

    # Separando anodiag
    HPET_menor['anodiag'] = pd.DatetimeIndex(HPET_menor['DT_SIN_PRI']).year

    # A partir de 2015 e UF RS
    HPET_menor = HPET_menor[(HPET_menor['anodiag'] > 2014) & (HPET_menor['SG_UF'] == '43')]

    idade_anos = np.where(HPET_menor["NU_IDADE_N"].astype(str).str.startswith("4"),
                        pd.to_numeric(HPET_menor["NU_IDADE_N"].astype(str).str[1:4]), 0)

    fx_etaria = pd.cut(HPET_menor.assign(idade_anos=idade_anos)["idade_anos"],
                    bins=[-np.inf, 1, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, np.inf],
                    labels=["<1", "1 a 4", "5 a 9", "10 a 14", "15 a 19", "20 a 29", "30 a 39", "40 a 49","50 a 59", "60 a 69", "70 a 79", "80>"])

    # Create a new DataFrame with two additional columns
    HPET_menor2 = HPET_menor.assign(idade_anos=idade_anos, fx_etaria=fx_etaria)

    # Adicione o código para filtrar casos de hepatite A, B e C por faixa etária
    Hepatites_fx_etaria = HPET_menor2.assign(
        Hep_A=np.where((HPET_menor2.ANTIHAVIGM == '1') | ((HPET_menor2.CLASSI_FIN == '2') & (HPET_menor2.CLAS_ETIOL == '01')), True, False),
        Hep_B= np.where((HPET_menor2.AGHBS == '1') | (HPET_menor2.ANTIHBCIGM == '1'), True, False),
        Hep_C = np.where((HPET_menor2.ANTIHCV == '1') | (HPET_menor2.TP_SOROHCV == '1'), True, False))


    # Adicione o código para contar casos de hepatite A por faixa etária
    conta_Hep_A_fx_etaria = Hepatites_fx_etaria[ Hepatites_fx_etaria['Hep_A'] == True].groupby(['anodiag', 'fx_etaria', 'CS_SEXO']).agg(total=('Hep_A', 'count')).reset_index()

    # Adicione o código para contar casos de hepatite B por faixa etária
    conta_Hep_B_fx_etaria = Hepatites_fx_etaria[ Hepatites_fx_etaria['Hep_B'] == True].groupby(['anodiag', 'fx_etaria', 'CS_SEXO']).agg(total=('Hep_B', 'count')).reset_index()

    # Adicione o código para contar casos de hepatite C por faixa etária
    conta_Hep_C_fx_etaria = Hepatites_fx_etaria[ Hepatites_fx_etaria['Hep_C'] == True].groupby(['anodiag', 'fx_etaria', 'CS_SEXO']).agg(total=('Hep_C', 'count')).reset_index()

    # Adicione o código para substituir "M" por "Masculino" e "F" por "Feminino" nas colunas de sexo
    conta_Hep_A_fx_etaria['CS_SEXO'] = conta_Hep_A_fx_etaria['CS_SEXO'].replace({'M': 'Masculino', 'F': 'Feminino'})
    conta_Hep_B_fx_etaria['CS_SEXO'] = conta_Hep_B_fx_etaria['CS_SEXO'].replace({'M': 'Masculino', 'F': 'Feminino'})
    conta_Hep_C_fx_etaria['CS_SEXO'] = conta_Hep_C_fx_etaria['CS_SEXO'].replace({'M': 'Masculino', 'F': 'Feminino'})

    # Adicione o código para converter a coluna "anodiag" em formato de data e criar uma nova coluna "Data_Formatada"
    conta_Hep_A_fx_etaria['anodiag'] = pd.to_datetime(conta_Hep_A_fx_etaria['anodiag'], format='%Y')
    conta_Hep_A_fx_etaria['Data_Formatada'] = conta_Hep_A_fx_etaria['anodiag'].dt.strftime('01/01/%Y')
    conta_Hep_B_fx_etaria['anodiag'] = pd.to_datetime(conta_Hep_B_fx_etaria['anodiag'], format='%Y')
    conta_Hep_B_fx_etaria['Data_Formatada'] = conta_Hep_B_fx_etaria['anodiag'].dt.strftime('01/01/%Y')
    conta_Hep_C_fx_etaria['anodiag'] = pd.to_datetime(conta_Hep_C_fx_etaria['anodiag'], format='%Y')
    conta_Hep_C_fx_etaria['Data_Formatada'] = conta_Hep_C_fx_etaria['anodiag'].dt.strftime('01/01/%Y')

    # Pegar o ano mínimo e máximo dos dados
    min_year_A = conta_Hep_A_fx_etaria['anodiag'].min().year
    max_year_A = conta_Hep_A_fx_etaria['anodiag'].max().year
    min_year_B = conta_Hep_B_fx_etaria['anodiag'].min().year
    max_year_B = conta_Hep_B_fx_etaria['anodiag'].max().year
    min_year_C = conta_Hep_C_fx_etaria['anodiag'].min().year
    max_year_C = conta_Hep_C_fx_etaria['anodiag'].max().year

    # Exibir contagem de casos por faixa etária para Hepatite A no período do dataset original
    print(f"Casos de Hepatite A ({min_year_A}-{max_year_A}) por faixa etária:")
    print(conta_Hep_A_fx_etaria.groupby('fx_etaria')['total'].sum())

    # Exibir contagem de casos por faixa etária para Hepatite B no período do dataset original
    print(f"\nCasos de Hepatite B ({min_year_B}-{max_year_B}) por faixa etária:")
    print(conta_Hep_B_fx_etaria.groupby('fx_etaria')['total'].sum())

    # Exibir contagem de casos por faixa etária para Hepatite C no período do dataset original
    print(f"\nCasos de Hepatite C ({min_year_C}-{max_year_C}) por faixa etária:")
    print(conta_Hep_C_fx_etaria.groupby('fx_etaria')['total'].sum())

except Exception as e:
    print(f"Ocorreu um erro durante a execução: {str(e)}")


Casos de Hepatite A (2015-2023) por faixa etária:
fx_etaria
<1           9
1 a 4       13
5 a 9       13
10 a 14     14
15 a 19     43
20 a 29    217
30 a 39    229
40 a 49    139
50 a 59    130
60 a 69     77
70 a 79     24
80>         17
Name: total, dtype: int64

Casos de Hepatite B (2015-2023) por faixa etária:
fx_etaria
<1           28
1 a 4        13
5 a 9         3
10 a 14      29
15 a 19     172
20 a 29    1277
30 a 39    2235
40 a 49    2698
50 a 59    2593
60 a 69    1546
70 a 79     537
80>         123
Name: total, dtype: int64

Casos de Hepatite C (2015-2023) por faixa etária:
fx_etaria
<1           200
1 a 4         19
5 a 9         18
10 a 14       68
15 a 19      350
20 a 29     1775
30 a 39     4710
40 a 49     8376
50 a 59    10028
60 a 69     6895
70 a 79     2691
80>          729
Name: total, dtype: int64


In [ ]:
# @title Salvando no Drive...
# Caminho para salvar os arquivos localmente
# Caminho para salvar os arquivos localmente
local_save_path = "/content/gdrive/MyDrive/Painel Hepatites Viras - Google Data Studio/Alimentação"
#
# Verificando se o diretório existe
if not os.path.exists(local_save_path):
    # Criando o diretório
    os.makedirs(local_save_path)

# Salvando os arquivos
file_name3 = f"DistribuiçãodeCasos_FxEtaria_A.xlsx"
file_path3 = os.path.join(local_save_path, file_name3)
conta_Hep_A_fx_etaria.to_excel(file_path3, index=False)

file_name4 = f"DistribuiçãodeCasos_FxEtaria_B.xlsx"
file_path4 = os.path.join(local_save_path, file_name4)
conta_Hep_B_fx_etaria.to_excel(file_path4, index=False)

file_name5 = f"DistribuiçãodeCasos_FxEtaria_C.xlsx"
file_path5 = os.path.join(local_save_path, file_name5)
conta_Hep_C_fx_etaria.to_excel(file_path5, index=False)